# wsgiref 源码解析
http://cizixs.com/2014/11/09/dive-into-wsgiref

Note: use python 2.7

wsgiref 是 PEP 333 定义的 wsgi 规范的范例实现，里面的功能包括了：

* 操作 wsgi 的环境变量
* 应答头部的处理
* 实现简单的 HTTP server
* 简单的对程序端和服务器端校验函数

Output

Output

## 源码分析

In [1]:
import wsgiref
help(wsgiref)

Help on package wsgiref:

NAME
    wsgiref - wsgiref -- a WSGI (PEP 3333) Reference Library

MODULE REFERENCE
    https://docs.python.org/3.5/library/wsgiref.html
    
    The following documentation is automatically generated from the Python
    source files.  It may be incomplete, incorrect or include features that
    are considered implementation detail and may vary between Python
    implementations.  When in doubt, consult the module reference at the
    location listed above.

DESCRIPTION
    Current Contents:
    
    * util -- Miscellaneous useful functions and wrappers
    
    * headers -- Manage response headers
    
    * handlers -- base classes for server/gateway implementations
    
    * simple_server -- a simple BaseHTTPServer that supports WSGI
    
    * validate -- validation wrapper that sits between an app and a server
      to detect errors in either
    
    To-Do:
    
    * cgi_gateway -- Run WSGI apps under CGI (pending a deployment standard)
    
    * cgi_

In [2]:
wsgiref.__path__

['/Users/binyang/anaconda/envs/py3.5/lib/python3.5/wsgiref']

In [3]:
!tree -I __pycache__ /Users/binyang/anaconda/envs/py3.5/lib/python3.5/wsgiref 

/Users/binyang/anaconda/envs/py3.5/lib/python3.5/wsgiref
├── __init__.py
├── handlers.py
├── headers.py
├── simple_server.py
├── util.py
└── validate.py

0 directories, 6 files


主要的代码结构如下图所示：

![](wsgiref_example/wsgiref_pic.jpg)

# simple_server.py

我们先看一下 make_server 是怎么启动一个 wsgi 服务器的：

这个函数做的事情就是：监听在本地的端口上，接受来自客户端的请求，通过 WSGIServer 和 WSGIRequestHandler 处理后，把请求交给程序的的可调用对象 app，然后返回 app 的结果给客户端。

这里有两个重要的类：WSGIServer 和 WSGIRequestHandler。下面分别看一下它们的代码和执行的功能。

### WSGIServer 
WSGIServer 在原来的 HTTPServer 上面封装了一层，在原来的 HTTPServer 的基础上又额外做了下面的事情：

* 覆写原来的 server_bind 函数，添加初始化 environ 变量的动作
* 添加了处理满足 wsgi 的 app 函数：set_app 和 get_app

### WSGIRequestHandler
这个类从名字就能知道它的功能——处理客户端的 HTTP 请求，它也是在原来处理 http 请求的BaseHTTPRequestHandler 类上添加了 wsgi 规范相关的内容。

* get_environ： 解析 environ 变量
* handle： 处理请求，把封装的环境变量交给 ServerHandler，然后由 ServerHandler 调用 wsgi app，ServerHandler 类会在下面介绍。

## handler.py

这个文件主要是 wsgi server 的处理过程，定义 start_response、调用 wsgi app 、处理 content-length 等等。

![](wsgiref_example/wsgi_2.jpg)

## 一条 HTTP 请求的旅程

服务器端启动服务，等到客户端输入 curl -i http://localhost:8000/ 命令，摁下回车键，看到终端上的输出，整个过程中，wsgi 的服务器端发生了什么呢？

1. 服务器程序创建 socket，并监听在特定的端口，等待客户端的连接
2. 客户端发送 http 请求
3. socket server 读取请求的数据，交给 http server
4. http server 根据 http 的规范解析请求，然后把请求交给 WSGIServer
5. WSGIServer 把客户端的信息存放在 environ 变量里，然后交给绑定的 handler 处理请求
6. HTTPHandler 解析请求，把 method、path 等放在 environ，然后 WSGIRequestHandler 把服务器端的信息也放到 environ 里
7. WSGIRequestHandler 调用绑定的 wsgi ServerHandler，把上面包含了服务器信息，客户端信息，本次请求信息得 environ 传递过去
8. wsgi ServerHandler 调用注册的 wsgi app，把 environ 和 start_response 传递过去
9. wsgi app 将reponse header、status、body 回传给 wsgi handler
10. 然后 handler 逐层传递，最后把这些信息通过 socket 发送到客户端
11. 客户端的程序接到应答，解析应答，并把结果打印出来。